In [55]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

import os
import re
import json
import glob
from collections import defaultdict
from textblob import TextBlob
from functools import partial
from pathlib import Path
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

import nltk
import spacy
# nlp = spacy.load('en_core_web_lg')
nlp=spacy.blank("en")
nlp.create_pipe('ner')
nlp.add_pipe('ner')
nlp.begin_training()
# spacy.prefer_gpu()
# spacy.require_gpu()
nlp.max_length = 1000000
from nltk.probability import FreqDist
from wordcloud import WordCloud, STOPWORDS

from tqdm.autonotebook import tqdm
import string
nlp=spacy.load("en_core_web_sm") 

# Getting the ner component
ner=nlp.get_pipe('ner')
# ner=nlp.get_pipe('ner')
%matplotlib inline

In [56]:
train_df = pd.read_csv('../train.csv')


In [57]:
train_files_path = 'C:\\Users\\gargk\\Documents\\coleridgeinitiative-show-us-the-data\\train'

In [58]:
def read_append_return(filename, train_files_path=train_files_path, output='text'):
    """
    Function to read json file and then return the text data from them and append to the dataframe
    """
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [59]:
tqdm.pandas()   #tqdm is used to show any code running with a progress bar. 
train_df['text'] = train_df['Id'].progress_apply(read_append_return)

In [60]:
train_df.head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label,text
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study used data from the National Educati...
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Dropping out of high school is not necessarily...
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,", stress satisfactory outcomes for all youth,..."
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Federal Reserve Bank of Richmond S1. Accountin...
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor ...


In [61]:
train_df['cleaned_label'].nunique()

130

In [62]:
train_df.head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label,text
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study used data from the National Educati...
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Dropping out of high school is not necessarily...
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,", stress satisfactory outcomes for all youth,..."
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Federal Reserve Bank of Richmond S1. Accountin...
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor ...


## Prep training data

In [63]:
def text_cleaning(text):
    '''
    Converts all text to lower case, Removes special charecters, emojis and multiple spaces
    text - Sentence that needs to be cleaned
    '''
    text = ''.join([k if k not in '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~' else ' ' for k in text ])
    text = re.sub('[^A-Za-z0-9.]+', ' ', str(text).lower()).strip()
#     text = re.sub("/'+/g", ' ', text)
    
    return text

In [64]:
%%time
tqdm.pandas()
train_df['text1'] = train_df['text'].progress_apply(text_cleaning)


Wall time: 1min 34s


In [65]:
def train_data_index_func(sentence, dataset):
    start = sentence.index(dataset)
    end = sentence.index(dataset) + len(dataset)
    dict1 = {"entities": [(start, end, "dataset")]}
    return dict1
    

def fill_train_data(sentences, dataset):
    for sentence in sentences:
        train_data.append((sentence, train_data_index_func(sentence, dataset)))

In [66]:

def find_sentence(txt, dataset, Id):
#     sentences = re.findall(r"([^.]*?"+dataset+"[^.]*\.)",txt)
    sentences = [sentence + '.' for sentence in txt.split('.') if dataset in sentence]
    if len(sentences) > 1: 
        dataset_sentence = [sentence for sentence in sentences if 'data' in sentence]
        if len(dataset_sentence)== 0: #If no keyword data
            fill_train_data(sentences, dataset)
            return sentences
        elif len(dataset_sentence)== 1: #If keyword data in only one of the sentence
            fill_train_data(dataset_sentence, dataset)
            return dataset_sentence
        else: #If keyword data in more than sentence
            dataset_sentence1 = [sentence for sentence in dataset_sentence if 'data set' in sentence]
            if len(dataset_sentence1)>=1: # If keyword 'data set' present
                fill_train_data(dataset_sentence1, dataset)
                return dataset_sentence1
            else: #If keyword 'data set' not present
                fill_train_data(dataset_sentence, dataset)
                return dataset_sentence
        
    else:
        fill_train_data(sentences, dataset)
        return sentences


In [67]:
train_data = []
train_df['sentences'] = train_df.progress_apply(lambda row: find_sentence(row['text1'], row['cleaned_label'], row['Id']), axis = 1)

## Training

In [68]:
LABEL = "dataset"


# Add the new label to ner
ner.add_label(LABEL)

# Resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [69]:
train_df.shape

(19661, 8)

In [70]:
# Sentences train data
train_data = []
# subset_train_df = train_df.sample(n=5000) # subset train df
subset_train_df = train_df.copy()
subset_train_df.progress_apply(lambda row: find_sentence(row['text1'], row['cleaned_label'], row['Id']), axis = 1)

0        [this study used data from the national educat...
1        [ using data on public and private school stud...
2        [ using the nationally representative longitud...
3        [ national education longitudinal study of 198...
4        [ using data from the national education longi...
5        [ in this study we employ base years and follo...
6        [ data are from the national education longitu...
7        [ these same students were resurveyed in 1990 ...
8        [ in the current study we examined the associa...
9        [ most notable are the studies based on three ...
10       [ this analysis is based on a large nationally...
11       [ the data presented here come from three nati...
12       [ analyzing data from the national education l...
13       [ to decompose the persisting gender gap in st...
14       [ 2 i find some evidence in support of this wi...
15       [ using data from the national education longi...
16       [ two national datasets the panel study of inc.

In [71]:
# # Whole text train data
# train_data = []
# def text_to_train(sentence, dataset):
#     try:
#         train_data.append((sentence, train_data_index_func(sentence, dataset)))
#     except: pass

# # subset_train_df = train_df.sample(n=10)
# subset_train_df = train_df.copy()
# subset_train_df.progress_apply(lambda row: text_to_train(row['text1'], row['cleaned_label']), axis = 1)

In [72]:
len(train_data)

34592

In [73]:
# Importing requirements
from spacy.util import minibatch, compounding
import random
from spacy.training import Example
# Import and load the spacy model
import spacy

# Begin training by disabling other pipeline components
# nlp.begin_training()

with nlp.disable_pipes(*other_pipes) :

    sizes = compounding(4.0, 32.0, 1.001)
# Training for 30 iterations     
    for itn in tqdm(range(30)):
        # shuffle examples before training
        random.shuffle(train_data)
        # batch up the examples using spaCy's minibatch
        batches = minibatch(train_data, size=sizes)
        # ictionary to store losses
        losses = {}
        for batch in tqdm(batches):
#             texts, annotations = zip(*batch)
#             # Calling update() over the iteration
#             nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
#             print("Losses", losses)
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
#             print("Losses", losses)

## Saving the model

In [74]:

output_dir=Path('../models3')

# Saving the model to the output directory
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta['name'] = 'my_ner0.1'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to ..\models3


## Loading the model

In [14]:
# Loading the model from the directory
output_dir=Path('../models')
print("Loading from",output_dir)
nlp2 = spacy.load(output_dir)
# move_names = list(ner.move_names)
# assert nlp2.get_pipe("ner").move_names == move_names
# doc2 = nlp2(test_text1)
# for ent in doc2.ents:
#     print(ent.label_, ent.text)

Loading from ..\models


## Testing

In [11]:
# subset_train_df['cleaned_label'].unique()

In [12]:
train_dataset_labels = ['baltimore longitudinal study of aging blsa ',
       'baltimore longitudinal study of aging', 'adni',
       'beginning postsecondary students longitudinal study',
       'sars cov 2 genome sequences',
       'optimum interpolation sea surface temperature',
       'sars cov 2 genome sequence', 'survey of doctorate recipients',
       'north american breeding bird survey bbs ', 'world ocean database',
       'alzheimer s disease neuroimaging initiative adni ',
       'early childhood longitudinal study',
       'survey of earned doctorates',
       'north american breeding bird survey', 'slosh model',
       'trends in international mathematics and science study',
       'nces common core of data', 'beginning postsecondary students',
       'national education longitudinal study',
       'baccalaureate and beyond', 'education longitudinal study',
       'ibtracs', 'agricultural resource management survey',
       'our world in data', 'census of agriculture',
       'genome sequences of sars cov 2', 'usda census of agriculture',
       'common core of data', 'noaa tide station',
       'nsf survey of earned doctorates', 'genome sequence of sars cov 2',
       'covid 19 open research dataset',
       'coastal change analysis program',
       'survey of graduate students and postdoctorates in science and engineering',
       'rural urban continuum codes', 'noaa c cap',
       'school survey on crime and safety',
       'covid 19 image data collection',
       'beginning postsecondary student',
       'national assessment of education progress', 'noaa tide gauge',
       'sars cov 2 full genome sequences',
       'the national institute on aging genetics of alzheimer s disease data storage site',
       'high school longitudinal study',
       'survey of industrial research and development',
       'nass census of agriculture',
       'alzheimers disease neuroimaging initiative',
       'baccalaureate and beyond longitudinal study',
       'higher education research and development survey',
       'international best track archive for climate stewardship',
       'nsf survey of industrial research and development',
       'coastal change analysis program land cover',
       'national center for education statistics common core of data',
       'noaa national water level observation network',
       'sars cov 2 complete genome sequences',
       'arms farm financial and crop production practices',
       'our world in data covid 19', 'genome sequences of covid 19',
       'noaa tidal station', 'covid 19 open research dataset cord 19 ',
       'genome sequence of covid 19',
       'agricultural resources management survey',
       'covid 19 open research data',
       'noaa optimum interpolation sea surface temperature',
       '2019 ncov genome sequence',
       'national water level observation network',
       'program for the international assessment of adult competencies',
       'covid 19 genome sequence',
       'complexity science hub covid 19 control strategies list cccsl ',
       'nsf ffrdc research and development survey',
       'national oceanic and atmospheric administration optimum interpolation sea surface temperature',
       '2019 ncov genome sequences',
       'national weather service nws storm surge risk',
       'national teacher and principal survey',
       'anss comprehensive earthquake catalog', 'covid 19 death data',
       'genome sequences of 2019 ncov', 'noaa world ocean database',
       'national science foundation survey of industrial research and development',
       'national science foundation survey of earned doctorates',
       'national science foundation survey of science and engineering research facilities',
       'rsna international covid 19 open radiology database',
       'anss comprehensive catalog',
       'nsf survey of science and engineering research facilities',
       'genome sequence of 2019 ncov',
       'nsf survey of graduate students and postdoctorates in science and engineering',
       'national science foundation higher education research and development survey',
       'characterizing health associated risks and your baseline disease in sars cov 2 charybdis ',
       'cas covid 19 antiviral candidate compounds dataset',
       'jh crown registry', 'covid 19 genome sequences',
       'advanced national seismic system anss comprehensive catalog comcat ',
       'national center for science and engineering statistics survey of earned doctorates',
       '2019 ncov complete genome sequences',
       'advanced national seismic system comprehensive catalog',
       'noaa sea lake and overland surges from hurricanes',
       'our world in data covid 19 dataset',
       'survey of science and engineering research facilities',
       'national center for science and engineering statistics survey of doctorate recipients',
       'usgs north american breeding bird survey',
       'complexity science hub covid 19 control strategies list'] 

In [13]:
train_df[~pd.DataFrame(train_df.cleaned_label.tolist()).isin(train_dataset_labels).values].tail(500)

,Id,pub_title,dataset_title,dataset_label,cleaned_label,text
436,93c27c16-9356-4b97-9785-9795ad9bb609,Statistical sources: Online resources for educ...,Aging Integrated Database (AGID),Aging Integrated Database (AGID),aging integrated database agid,here are myriad statistical sources available ...
508,93c27c16-9356-4b97-9785-9795ad9bb609,Statistical sources: Online resources for educ...,Aging Integrated Database (AGID),Aging Integrated Database,aging integrated database,here are myriad statistical sources available ...
519,0eb834f3-1be2-4756-9a88-572e752ff057,Comparison of Rip Current Hazard Likelihood Fo...,NOAA Tide Gauge,NOAA water level station,noaa water level station,Although rip currents are a major hazard for b...
520,431e3576-bcfd-4917-9730-cdff9cf03be8,Influence of atmospheric and sea-surface corre...,NOAA Tide Gauge,NOAA water level station,noaa water level station,Hyperspectral instruments provide the spectral...
1207,9fe28488-002a-4e2f-93a2-e64d31cccd26,Mangrove canopy height globally related to pre...,International Best Track Archive for Climate S...,NOAA International Best Track Archive for Clim...,noaa international best track archive for clim...,", storing up to four times as much carbon per ..."
1450,aa434be4-4725-4920-ad79-170efec7c527,Research Funding,Higher Education Research and Development Survey,NSF Higher Education Research and Development ...,nsf higher education research and development ...,The importance of research at higher education...
1472,11cd3b53-957d-4eb0-bcdf-652d34e63af4,The Rise of Big Data in Neurorehabilitation.,The National Institute on Aging Genetics of Al...,The National Institute on Aging Genetics of Al...,the national institute on aging genetics of al...,"In some fields, Big Data has been instrumental..."
1473,d6f180b7-fe12-49a2-961f-7ad35413c638,Whole genome sequencing of Caribbean Hispanic ...,The National Institute on Aging Genetics of Al...,The National Institute on Aging Genetics of Al...,the national institute on aging genetics of al...,a Consortium authors are listed in Acknowledge...
1474,cf6220b1-b1a5-408f-8560-4da31a941751,Genome-wide linkage analyses of non-Hispanic w...,The National Institute on Aging Genetics of Al...,The National Institute on Aging Genetics of Al...,the national institute on aging genetics of al...,Introduction: Few high penetrance variants tha...
1475,7b6684d8-67eb-4713-a888-1c54b58c19d4,Identification of Novel Alzheimer’s Disease Lo...,The National Institute on Aging Genetics of Al...,The National Institute on Aging Genetics of Al...,the national institute on aging genetics of al...,With the advent of whole genome-sequencing (WG...


In [14]:
# print("Loading from", output_dir)
# nlp2 = spacy.load(output_dir)
# assert nlp2.get_pipe("ner").move_names == move_names

In [15]:
test = train_df.loc[14189]
test_text1 = test['text1']
doc2 = nlp2(test_text1)

KeyError: 'text1'

In [16]:
test['cleaned_label']

'noaa storm surge inundation'

In [26]:
for ent in doc2.ents:
    print(ent.label_, ent.text)

dataset slosh model
dataset coastal change hazards
dataset coastal change science along
dataset coastal change hazards
dataset coastal observation station


In [32]:
print( ent.text)

coastal observation station


In [27]:
test_df = train_df[~pd.DataFrame(train_df.cleaned_label.tolist()).isin(train_dataset_labels).values].tail(500)

In [28]:
test_df.shape

(65, 8)

In [58]:
def predict_label(Id, dataset_title, cleaned_label, text1):
    try:
        doc2 = nlp2(text1)
        pred_labels.append([ent.text for ent in doc2.ents])
        loc = len(pred_labels)
        Ids.append([Id for i in range(len(pred_labels[loc-1]))])
        dataset_titles.append([dataset_title for i in range(len(pred_labels[loc-1]))])
        cleaned_labels.append([cleaned_label for i in range(len(pred_labels[loc-1]))])
        text1s.append([text1 for i in range(len(pred_labels[loc-1]))])
    except Exception as e: print(e)
#     pred_df = 


In [59]:
pred_labels = []
Ids = []
dataset_titles = []
cleaned_labels = []
text1s = []

test_df.progress_apply(lambda row: predict_label(row['Id'], row['dataset_title'], 
                                                 row['cleaned_label'], row['text1']), axis = 1)

[E088] Text of length 2667496 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.
[E088] Text of length 2667496 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.
[E088] Text of length 2667496 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 char

436      None
508      None
519      None
520      None
1207     None
1450     None
1472     None
1473     None
1474     None
1475     None
1476     None
1477     None
1478     None
1479     None
1480     None
7085     None
8071     None
11557    None
11586    None
11587    None
12651    None
13328    None
14189    None
14338    None
14348    None
14536    None
14801    None
14806    None
14810    None
14811    None
14868    None
14870    None
14872    None
14888    None
15190    None
15270    None
15310    None
15343    None
15403    None
15473    None
15483    None
15486    None
15488    None
15491    None
15618    None
15777    None
16069    None
16690    None
18480    None
18490    None
18491    None
18722    None
18723    None
18724    None
18725    None
18726    None
19154    None
19155    None
19156    None
19169    None
19170    None
19653    None
19656    None
19659    None
19660    None
dtype: object

In [69]:
pred_labels1 = [item for sublist in pred_labels for item in sublist]
Ids1 = [item for sublist in Ids for item in sublist]
dataset_titles1 = [item for sublist in dataset_titles for item in sublist]
cleaned_labels1 = [item for sublist in cleaned_labels for item in sublist]
text1s1 = [item for sublist in text1s for item in sublist]

In [71]:
len(Ids1)

175

In [72]:
pred_df = pd.DataFrame(list(zip(Ids1, dataset_titles1, cleaned_labels1, text1s1, pred_labels1)),
               columns =[ 'Id', 'dataset_title', 'cleaned_label', 'text', 'pred_labels'])

In [76]:
pred_df

,Id,dataset_title,cleaned_label,text,pred_labels
0,93c27c16-9356-4b97-9785-9795ad9bb609,Aging Integrated Database (AGID),aging integrated database agid,here are myriad statistical sources available ...,international archive of education data
1,93c27c16-9356-4b97-9785-9795ad9bb609,Aging Integrated Database (AGID),aging integrated database agid,here are myriad statistical sources available ...,education the open doors
2,93c27c16-9356-4b97-9785-9795ad9bb609,Aging Integrated Database (AGID),aging integrated database,here are myriad statistical sources available ...,international archive of education data
3,93c27c16-9356-4b97-9785-9795ad9bb609,Aging Integrated Database (AGID),aging integrated database,here are myriad statistical sources available ...,education the open doors
4,0eb834f3-1be2-4756-9a88-572e752ff057,NOAA Tide Gauge,noaa water level station,although rip currents are a major hazard for b...,noaa water level station
5,431e3576-bcfd-4917-9730-cdff9cf03be8,NOAA Tide Gauge,noaa water level station,hyperspectral instruments provide the spectral...,noaa water level station
6,9fe28488-002a-4e2f-93a2-e64d31cccd26,International Best Track Archive for Climate S...,noaa international best track archive for clim...,storing up to four times as much carbon per un...,coastal ocean with
7,9fe28488-002a-4e2f-93a2-e64d31cccd26,International Best Track Archive for Climate S...,noaa international best track archive for clim...,storing up to four times as much carbon per un...,noaa international best track archive for clim...
8,9fe28488-002a-4e2f-93a2-e64d31cccd26,International Best Track Archive for Climate S...,noaa international best track archive for clim...,storing up to four times as much carbon per un...,ibtracs
9,aa434be4-4725-4920-ad79-170efec7c527,Higher Education Research and Development Survey,nsf higher education research and development ...,the importance of research at higher education...,higher education research and development


In [77]:
pred_df.shape

(175, 5)

In [75]:
pred_df[pred_df['cleaned_label'] == pred_df['pred_labels']].shape

(20, 5)

In [ ]:
sample_sub = pd.read_csv('../sample_submission.csv')


In [ ]:
%%time
tqdm.pandas()
test_files_path = 'C:\\Users\\gargk\\Documents\\coleridgeinitiative-show-us-the-data\\test'
sample_sub['text'] = sample_sub['Id'].progress_apply(partial(read_append_return, train_files_path=test_files_path))